<a href="https://colab.research.google.com/github/radonys/webscrapers/blob/master/Twitter_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter Scraper

## Install Modules

In [0]:
!pip install tweepy

## Import Modules

In [0]:
import tweepy
from tweepy import OAuthHandler

import json
import pandas as pd
import re
import os
import requests
import shutil

from google.colab import files

## Variable Declarations

In [0]:
twitter_handles = ['AmSigFurniture', 'AshleyHomeStore', 'Bonobos', 'CocaCola', 'IKEAUSA', 'lazboy', 'potterybarn', 'raymourflanigan', 'SimplyGum', 'Star_Furniture', 'WilliamsSonoma']

## Twitter Credentials

References:

1) How to Scrape Data from Twitter Using Python ([PromptCloud](https://www.promptcloud.com/blog/scrape-twitter-data-using-python-r/))

In [0]:
print('Please upload the JSON file with Twitter Application Credentials')
#files.upload()

In [0]:
twitter_cred = dict()

twitter_cred['CONSUMER_KEY'] = ''
twitter_cred['CONSUMER_SECRET'] = ''
twitter_cred['ACCESS_KEY'] = ''
twitter_cred['ACCESS_SECRET'] = ''

'''with open('twitter_credentials.json', 'r') as protected:
  json.dump(twitter_cred, protected, sort_keys=True)'''

authentication = OAuthHandler(twitter_cred['CONSUMER_KEY'], twitter_cred['CONSUMER_SECRET'])
authentication.set_access_token(twitter_cred['ACCESS_KEY'], twitter_cred['ACCESS_SECRET'])

twitter_api = tweepy.API(authentication)

## Helper Functions

### Retrieve Twitter Media Data & URL

References:

1) How to Scrape Data from Twitter Using Python ([PromptCloud](https://www.promptcloud.com/blog/scrape-twitter-data-using-python-r/))

In [0]:
def scrape_twitter_media(handle):
  
  global twitter_api
  
  tweets = twitter_api.user_timeline(screen_name=handle, count=200)
  
  last_tweet_id = tweets[len(tweets)-1].id
  
  flag = True
  
  while flag:
    
    next_tweets = twitter_api.user_timeline(screen_name=handle, count=200, max_id=last_tweet_id - 1)
    
    if len(next_tweets)==0:
      flag = False
    
    else:
      
      tweets = tweets + next_tweets
      last_tweet_id = tweets[len(tweets)-1].id
    
  return tweets

### Select Relevant Information

References:

1) How to Scrape Data from Twitter Using Python ([PromptCloud](https://www.promptcloud.com/blog/scrape-twitter-data-using-python-r/))

2) Convert Tweepy Status object into JSON ([StackOverFlow](https://stackoverflow.com/questions/27900451/convert-tweepy-status-object-into-json))

3) Convert string to JSON using Python ([StackOverFlow](https://stackoverflow.com/questions/4528099/convert-string-to-json-using-python))

In [0]:
def select_relevant_information(tweet):
  
  temp = dict()
  key_values = ['id', 'created_at', 'id_str', 'retweet_count', 'favorite_count', 'user']
  
  media = tweet.entities.get('media', [])
  
  if len(media)>0:
    
    temp['media_url'] = [media[0]['media_url']]
    json_data = json.loads(json.dumps(tweet._json))
    
    for key in key_values:
      
      if key=='user':
        temp[key] = [json_data[key]['screen_name']]
        
      else:
        temp[key] = [json_data[key]]
        
    return temp
    
  else:
    
    return None
    

### Save Metadata

In [0]:
def save_metadata(tweets, save_name):
  
  data = pd.DataFrame()
  
  for tweet in tweets:
    
    temp_dict = select_relevant_information(tweet)
    
    if temp_dict!=None:
      
      tweet_data = pd.DataFrame.from_dict(temp_dict, orient='columns')
      data = data.append(tweet_data)
      
  data.index = range(len(data.index))
  data.to_csv(handle+'_data.csv')
  save_images(data, handle+'_images')

### Save Images

References:

1) Scraping Instagram with Python (using Selenium and Beautiful Soup) ([Medium](https://medium.com/@srujana.rao2/scraping-instagram-with-python-using-selenium-and-beautiful-soup-8b72c186a058))

In [0]:
def save_images(data, save_name):
  
  images_path = save_name+'/'
  
  if not os.path.isdir(images_path):
    os.mkdir(images_path)
  
  for i in range(0, len(data['media_url'])):
    
    image = requests.get(data['media_url'][i])
    
    with open(images_path + data['id_str'][i] + ".jpg", 'wb') as file:
      file.write(image.content)

## Scrape Twitter Media

In [0]:
for handle in twitter_handles:
  
  tweets = scrape_twitter_media(handle)
  print(len(tweets))
  save_metadata(tweets, handle)
  shutil.make_archive(handle+'_images', 'zip', handle+'_images')